In [ ]:
%matplotlib inline
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import minmax_scale
from sklearn.decomposition import PCA


In [ ]:
#carrega o dataset
dados = pd.read_csv('/home/erasmor/Downloads/2017/reduzido_total_sem_baixa_1.csv',sep=",",encoding = 'utf-8',  header=0,na_values='.',dtype={'Label':'category'})
#remove valores infinitos
dados.replace(-np.Inf, np.nan)

#substitui valores NaN
dados.fillna(dados.mean())


In [ ]:
#mostra como está a base de dados
#dados.head()
dados.dtypes

In [ ]:
dados.memory_usage(deep=True)

In [ ]:
# verifica quantas instâncias (linhas) e quantos atributos (colunas) a base de dados contém
print("numero de linhas e colunas: ",dados.shape)

In [ ]:
#visualizar distribições por classes contidas no csv - informar nome da classe alvo
print(dados.groupby('Label').size())

In [ ]:
#X_raw_normalize = MinMaxScaler(X_raw_normalize.reshape(0, 1)).reshape(len(X_raw_normalize))
#X_raw_normalizetd2 = (X_raw_normalize - X_raw_normalize.min(axis=0)) / (X_raw_normalize.max(axis=0) - X_raw_normalize.min(axis=0))
# Obtendo os nomes das colunas do DataFrame como uma lista.
cols = list(dados.columns)
# colunas que nao serao normalizadas
cols.remove('Label')
# Copiando os dados e aplicando a normalizacao por reescala nas colunas do DataFrame que contem
# valores continuos. Por padrao, o metodo minmax_scale reescala com min=0 e max=1.
dados = dados[~dados.isin([np.nan, np.inf, -np.inf]).any(1)]
dados[cols] = dados[cols].apply(minmax_scale)


In [ ]:
#define as colunas de atributos e a coluna da classe (de 0 a 72 são atributos e após a 72 é a classe)
# "X_raw" é features/atributos e "y_raw" é target/classe ==> As duas formas abaixo dão certo.
#array = dataset.values
#X_raw = array[:,0:72]
#y_raw = array[:,72]
X_raw = dados.iloc[:, :-1].values # atributos
y_raw = dados.iloc[:, 78].values # classe de ataques
X_raw = np.nan_to_num(X_raw.astype(np.float32))


In [ ]:
#transformar a variável Y com valores categóricos das classses de ataques em valores:
labelencoder_y = LabelEncoder()
y_raw = labelencoder_y.fit_transform(y_raw)
mapping = dict(zip(labelencoder_y.classes_, range(1, len(labelencoder_y.classes_)+1)))

print(mapping)

In [ ]:
# Instanciando um PCA. O parametro n_components indica a quantidade de dimensoes que a base
# original sera reduzida.
pca = PCA(n_components=10, whiten=True,random_state=42)

In [ ]:
# Aplicando o pca na base de dados. O atributo 'values' retorna um numpy.array
# de duas dimensões (matriz) contendo apenas os valores numericos do DataFrame.
X_raw = pca.fit_transform(X_raw)

In [ ]:
def committee_sampling_knn(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial):
    
    from sklearn.neural_network import MLPClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
    import functools
    from IPython.display import clear_output
    from modAL.uncertainty import classifier_uncertainty
    from modAL.models import ActiveLearner, Committee
    from modAL.uncertainty import uncertainty_sampling
    from modAL.disagreement import vote_entropy_sampling
    from functools import partial
    
    
        
    #define nome de arquivos para salvar
    indica_pool=str(idx_dobra)
    
    # recupera as amostras de treino iniciais - a extratificação realizada só serve para tal finalidade.
    # No caso força a buscar pelo menos uma amostras de cada rótulo disponível (train_size= len(np.unique(y_raw)).
    # Realizar a busca aleatoriamente não garante iniciar com uma instância de cada classe.
    X_train_inicial, X_test_inicial, y_train_inicial, y_test_inicial = train_test_split(X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]], train_size= len(np.unique(y_raw[idx_data[idx_dobra][TRAIN]])) + t_inicial, random_state=42, stratify = y_raw[idx_data[idx_dobra][TRAIN]])
    #print("tamanho de X_train inicial: ",X_train_inicial.shape," tamanho de y_train inicial: ",y_train_inicial.shape)
    #print(y_train_inicial)  
    # recupera amostras de teste de acordo com a dobra em uso
    X_teste, y_teste = X_raw[idx_data[idx_dobra][TEST]], y_raw[idx_data[idx_dobra][TEST]]
    # recupera amostras de treino (será o pool) de acordo com a dobra em uso
    X_pool, y_pool = X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]]
    
    #isola exemplos rotulados para o treinamento inicial
    X_train = X_train_inicial
    y_train = y_train_inicial
    
    # initializing Committee members
    n_members = 3
    learner_list = list()
    
    #instancia classificadores de aprendizado ativo
    preset_batch = partial(vote_entropy_sampling,n_instances=BATCH_SIZE)
    learner_knn = ActiveLearner(estimator=KNeighborsClassifier(n_neighbors=3),X_training=X_train, y_training=y_train)
    learner_list.append(learner_knn)
    arquivo_accuracy_knn = open("committee_accuracy_performance_knn_dobra_"+indica_pool+".txt","a")
    arquivo_accuracy_history_knn = ("committee_accuracy_history_knn_dobra_"+indica_pool+".csv")
    arquivo_f1_score_history_knn = ("committee_F1_Score_history_knn_dobra_"+indica_pool+".csv")
    arquivo_tempo_history_knn = ("committee_tempo_history_knn_dobra_"+indica_pool+".csv")
              
    # assembling the committee
    committee = Committee(learner_list=learner_list,query_strategy=preset_batch)
    
    #Registro da pontuação na porção de teste com o treinamento inicial
    performance_history_knn=[]
    f1_score_history_knn=[]
    tempo_history_knn=[]
    uncertain_sample_score_knn = committee.score(X_teste, y_teste)
    performance_history_knn.append(uncertain_sample_score_knn)
    predictions = committee.predict(X_teste)
    f1score_knn = 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
    f1_score_history_knn.append(f1score_knn)
    
        
    for index in range(N_QUERIES):
        #inicia calculo do tempo de processamento da estratégia (consulta e inserção)
        t1_knn = time.time()
        #recupera amostras do pool baseado na estratégia de consulta
        query_index, query_instance = committee.query(X_pool)
            
        # Ensina ao modelo ActiveLearner o registro solicitado (amostras vão para o topo).
        committee.teach(X=X_pool[query_index].reshape(BATCH_SIZE, -1),y=y_pool[query_index].reshape(BATCH_SIZE, ))
       
        # apaga os modelos consultados
        X_pool = np.delete(X_pool, query_index, axis=0)
        y_pool = np.delete(y_pool, query_index)
        
        #termina calculo de processamentoda estratégia
        t2_knn = time.time()
        time_elapsed_knn = (t2_knn-t1_knn)
        hours_knn, rem_knn = divmod(time_elapsed_knn, 3600)
        minutes_knn, seconds_knn = divmod(rem_knn, 60)
        tempo_history_knn.append("{:0>2}:{:0>2}:{:0>2}".format(int(hours_knn),int(minutes_knn),int(seconds_knn)))
        
        # verifica a performance após a inclusão de dados novos 
        committee_sample_score_knn = committee.score(X_teste, y_teste)
        predictions = committee.predict(X_teste)
        clear_output(wait=True)
        print('Accuracy KNN after query no. %d: %f' % (index+1, committee_sample_score_knn))
        arquivo_accuracy_knn.write('Accuracy after query no. %d: %f \n' % (index+1,committee_sample_score_knn))
        performance_history_knn.append(committee_sample_score_knn)
        #print ('Accuracy after query no. %d: %f' % (index+1, accuracy_score(y_test, predictions)))
        #arquivo_performance_knn.write('Accuracy after query no. %d: %f \n' % (index+1,accuracy_score(y_teste, predictions)))
        #print ('Precision after query no. %d: %f' % (index+1, precision_score(y_test, predictions,average='macro',zero_division=1)))
        arquivo_accuracy_knn.write('Precision after query no. %d: %f \n' % (index+1,precision_score(y_teste, predictions,average='macro',zero_division=1)))
        #print ('Recall after query no. %d: %f' % (index+1, recall_score(y_test, predictions, average='macro',zero_division=1)))
        arquivo_accuracy_knn.write('Recall after query no. %d: %f \n' % (index+1, recall_score(y_teste, predictions, average='macro',zero_division=1)))
        #print ('F1 score after query no. %d: %f' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
        f1score_knn= 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
        arquivo_accuracy_knn.write('F1 score after query no. %d: %f \n' % (index+1, f1score_knn))
        f1_score_history_knn.append(f1score_knn)
        #print ("========================================")
        arquivo_accuracy_knn.write('======================================== \n')
                
    arquivo_accuracy_knn.write("\n Classification report - KNN \n")
    arquivo_accuracy_knn.write("\n Avaliação por classe \n")
    arquivo_accuracy_knn.write(classification_report(y_teste, predictions,zero_division=1))
    np.savetxt(arquivo_accuracy_history_knn, performance_history_knn,delimiter=",")
    np.savetxt(arquivo_f1_score_history_knn, f1_score_history_knn)
    np.savetxt(arquivo_tempo_history_knn,tempo_history_knn,fmt="%s")
    arquivo_accuracy_knn.close()

def committee_sampling_rf(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial):
    
    from sklearn.neural_network import MLPClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
    import functools
    from IPython.display import clear_output
    from modAL.uncertainty import classifier_uncertainty
    from modAL.models import ActiveLearner, Committee
    from modAL.uncertainty import uncertainty_sampling
    from modAL.disagreement import vote_entropy_sampling
    from functools import partial
    
    
        
    #define nome de arquivos para salvar
    indica_pool=str(idx_dobra)
    
    # recupera as amostras de treino iniciais - a extratificação realizada só serve para tal finalidade.
    # No caso força a buscar pelo menos uma amostras de cada rótulo disponível (train_size= len(np.unique(y_raw)).
    # Realizar a busca aleatoriamente não garante iniciar com uma instância de cada classe.
    X_train_inicial, X_test_inicial, y_train_inicial, y_test_inicial = train_test_split(X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]], train_size= len(np.unique(y_raw[idx_data[idx_dobra][TRAIN]])) + t_inicial, random_state=42, stratify = y_raw[idx_data[idx_dobra][TRAIN]])
    #print("tamanho de X_train inicial: ",X_train_inicial.shape," tamanho de y_train inicial: ",y_train_inicial.shape)
    #print(y_train_inicial)  
    # recupera amostras de teste de acordo com a dobra em uso
    X_teste, y_teste = X_raw[idx_data[idx_dobra][TEST]], y_raw[idx_data[idx_dobra][TEST]]
    # recupera amostras de treino (será o pool) de acordo com a dobra em uso
    X_pool, y_pool = X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]]
    
    #isola exemplos rotulados para o treinamento inicial
    X_train = X_train_inicial
    y_train = y_train_inicial
    
    # initializing Committee members
    n_members = 3
    learner_list = list()
    
    #instancia classificadores de aprendizado ativo
    preset_batch = partial(vote_entropy_sampling,n_instances=BATCH_SIZE)
    learner_rf = ActiveLearner(estimator=RandomForestClassifier(n_estimators=15,random_state=42),X_training=X_train, y_training=y_train)
    learner_list.append(learner_rf)
    arquivo_accuracy_rf = open("committee_accuracy_performance_rf_dobra_"+indica_pool+".txt","a")
    arquivo_accuracy_history_rf = ("committee_accuracy_history_rf_dobra_"+indica_pool+".csv")
    arquivo_f1_score_history_rf = ("committee_F1_Score_history_rf_dobra_"+indica_pool+".csv")
    arquivo_tempo_history_rf = ("committee_tempo_history_rf_dobra_"+indica_pool+".csv")
              
    # assembling the committee
    committee = Committee(learner_list=learner_list,query_strategy=preset_batch)
    
    #Registro da pontuação na porção de teste com o treinamento inicial
    performance_history_rf=[]
    f1_score_history_rf=[]
    tempo_history_rf=[]
    uncertain_sample_score_rf = committee.score(X_teste, y_teste)
    performance_history_rf.append(uncertain_sample_score_rf)
    predictions = committee.predict(X_teste)
    f1score_rf = 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
    f1_score_history_rf.append(f1score_rf)
    
        
    for index in range(N_QUERIES):
        #inicia calculo do tempo de processamento da estratégia (consulta e inserção)
        t1_rf = time.time()
        #recupera amostras do pool baseado na estratégia de consulta
        query_index, query_instance = committee.query(X_pool)
            
        # Ensina ao modelo ActiveLearner o registro solicitado (amostras vão para o topo).
        committee.teach(X=X_pool[query_index].reshape(BATCH_SIZE, -1),y=y_pool[query_index].reshape(BATCH_SIZE, ))
       
        # apaga os modelos consultados
        X_pool = np.delete(X_pool, query_index, axis=0)
        y_pool = np.delete(y_pool, query_index)
        
        #termina calculo de processamentoda estratégia
        t2_rf = time.time()
        time_elapsed_rf = (t2_rf-t1_rf)
        hours_rf, rem_rf = divmod(time_elapsed_rf, 3600)
        minutes_rf, seconds_rf = divmod(rem_rf, 60)
        tempo_history_rf.append("{:0>2}:{:0>2}:{:0>2}".format(int(hours_rf),int(minutes_rf),int(seconds_rf)))
        
        # verifica a performance após a inclusão de dados novos 
        committee_sample_score_rf = committee.score(X_teste, y_teste)
        predictions = committee.predict(X_teste)
        clear_output(wait=True)
        print('Accuracy RF after query no. %d: %f' % (index+1, committee_sample_score_rf))
        arquivo_accuracy_rf.write('Accuracy after query no. %d: %f \n' % (index+1,committee_sample_score_rf))
        performance_history_rf.append(committee_sample_score_rf)
        #print ('Accuracy after query no. %d: %f' % (index+1, accuracy_score(y_test, predictions)))
        #arquivo_performance_rf.write('Accuracy after query no. %d: %f \n' % (index+1,accuracy_score(y_teste, predictions)))
        #print ('Precision after query no. %d: %f' % (index+1, precision_score(y_test, predictions,average='macro',zero_division=1)))
        arquivo_accuracy_rf.write('Precision after query no. %d: %f \n' % (index+1,precision_score(y_teste, predictions,average='macro',zero_division=1)))
        #print ('Recall after query no. %d: %f' % (index+1, recall_score(y_test, predictions, average='macro',zero_division=1)))
        arquivo_accuracy_rf.write('Recall after query no. %d: %f \n' % (index+1, recall_score(y_teste, predictions, average='macro',zero_division=1)))
        #print ('F1 score after query no. %d: %f' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
        f1score_rf= 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
        arquivo_accuracy_rf.write('F1 score after query no. %d: %f \n' % (index+1, f1score_rf))
        f1_score_history_rf.append(f1score_rf)
        #print ("========================================")
        arquivo_accuracy_rf.write('======================================== \n')
                
    arquivo_accuracy_rf.write("\n Classification report - RF \n")
    arquivo_accuracy_rf.write("\n Avaliação por classe \n")
    arquivo_accuracy_rf.write(classification_report(y_teste, predictions,zero_division=1))
    np.savetxt(arquivo_accuracy_history_rf, performance_history_rf,delimiter=",")
    np.savetxt(arquivo_f1_score_history_rf, f1_score_history_rf)
    np.savetxt(arquivo_tempo_history_rf,tempo_history_rf,fmt="%s")
    arquivo_accuracy_rf.close()
    
    
def committee_sampling_tree(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial):
    
    from sklearn.neural_network import MLPClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
    import functools
    from IPython.display import clear_output
    from modAL.uncertainty import classifier_uncertainty
    from modAL.models import ActiveLearner, Committee
    from modAL.uncertainty import uncertainty_sampling
    from modAL.disagreement import vote_entropy_sampling
    from functools import partial
    
    
        
    #define nome de arquivos para salvar
    indica_pool=str(idx_dobra)
    
    # recupera as amostras de treino iniciais - a extratificação realizada só serve para tal finalidade.
    # No caso força a buscar pelo menos uma amostras de cada rótulo disponível (train_size= len(np.unique(y_raw)).
    # Realizar a busca aleatoriamente não garante iniciar com uma instância de cada classe.
    X_train_inicial, X_test_inicial, y_train_inicial, y_test_inicial = train_test_split(X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]], train_size= len(np.unique(y_raw[idx_data[idx_dobra][TRAIN]])) + t_inicial, random_state=42, stratify = y_raw[idx_data[idx_dobra][TRAIN]])
    #print("tamanho de X_train inicial: ",X_train_inicial.shape," tamanho de y_train inicial: ",y_train_inicial.shape)
    #print(y_train_inicial)  
    # recupera amostras de teste de acordo com a dobra em uso
    X_teste, y_teste = X_raw[idx_data[idx_dobra][TEST]], y_raw[idx_data[idx_dobra][TEST]]
    # recupera amostras de treino (será o pool) de acordo com a dobra em uso
    X_pool, y_pool = X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]]
    
    #isola exemplos rotulados para o treinamento inicial
    X_train = X_train_inicial
    y_train = y_train_inicial
    
    # initializing Committee members
    n_members = 3
    learner_list = list()
    
    #instancia classificadores de aprendizado ativo
    preset_batch = partial(vote_entropy_sampling,n_instances=BATCH_SIZE)
    learner_tree = ActiveLearner(estimator=DecisionTreeClassifier(),X_training=X_train, y_training=y_train)
    learner_list.append(learner_tree)
    arquivo_accuracy_tree = open("committee_accuracy_performance_tree_dobra_"+indica_pool+".txt","a")
    arquivo_accuracy_history_tree = ("committee_accuracy_history_tree_dobra_"+indica_pool+".csv")
    arquivo_f1_score_history_tree = ("committee_F1_Score_history_tree_dobra_"+indica_pool+".csv")
    arquivo_tempo_history_tree = ("committee_tempo_history_tree_dobra_"+indica_pool+".csv")
              
    # assembling the committee
    committee = Committee(learner_list=learner_list,query_strategy=preset_batch)
    
    #Registro da pontuação na porção de teste com o treinamento inicial
    performance_history_tree=[]
    f1_score_history_tree=[]
    tempo_history_tree=[]
    uncertain_sample_score_tree = committee.score(X_teste, y_teste)
    performance_history_tree.append(uncertain_sample_score_tree)
    predictions = committee.predict(X_teste)
    f1score_tree = 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
    f1_score_history_tree.append(f1score_tree)
    
        
    for index in range(N_QUERIES):
        #inicia calculo do tempo de processamento da estratégia (consulta e inserção)
        t1_tree = time.time()
        #recupera amostras do pool baseado na estratégia de consulta
        query_index, query_instance = committee.query(X_pool)
            
        # Ensina ao modelo ActiveLearner o registro solicitado (amostras vão para o topo).
        committee.teach(X=X_pool[query_index].reshape(BATCH_SIZE, -1),y=y_pool[query_index].reshape(BATCH_SIZE, ))
       
        # apaga os modelos consultados
        X_pool = np.delete(X_pool, query_index, axis=0)
        y_pool = np.delete(y_pool, query_index)
        
        #termina calculo de processamentoda estratégia
        t2_tree = time.time()
        time_elapsed_tree = (t2_tree-t1_tree)
        hours_tree, rem_tree = divmod(time_elapsed_tree, 3600)
        minutes_tree, seconds_tree = divmod(rem_tree, 60)
        tempo_history_tree.append("{:0>2}:{:0>2}:{:0>2}".format(int(hours_tree),int(minutes_tree),int(seconds_tree)))
        
        # verifica a performance após a inclusão de dados novos 
        committee_sample_score_tree = committee.score(X_teste, y_teste)
        predictions = committee.predict(X_teste)
        clear_output(wait=True)
        print('Accuracy TREE after query no. %d: %f' % (index+1, committee_sample_score_tree))
        arquivo_accuracy_tree.write('Accuracy after query no. %d: %f \n' % (index+1,committee_sample_score_tree))
        performance_history_tree.append(committee_sample_score_tree)
        #print ('Accuracy after query no. %d: %f' % (index+1, accuracy_score(y_test, predictions)))
        #arquivo_performance_tree.write('Accuracy after query no. %d: %f \n' % (index+1,accuracy_score(y_teste, predictions)))
        #print ('Precision after query no. %d: %f' % (index+1, precision_score(y_test, predictions,average='macro',zero_division=1)))
        arquivo_accuracy_tree.write('Precision after query no. %d: %f \n' % (index+1,precision_score(y_teste, predictions,average='macro',zero_division=1)))
        #print ('Recall after query no. %d: %f' % (index+1, recall_score(y_test, predictions, average='macro',zero_division=1)))
        arquivo_accuracy_tree.write('Recall after query no. %d: %f \n' % (index+1, recall_score(y_teste, predictions, average='macro',zero_division=1)))
        #print ('F1 score after query no. %d: %f' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
        f1score_tree= 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
        arquivo_accuracy_tree.write('F1 score after query no. %d: %f \n' % (index+1, f1score_tree))
        f1_score_history_tree.append(f1score_tree)
        #print ("========================================")
        arquivo_accuracy_tree.write('======================================== \n')
                
    arquivo_accuracy_tree.write("\n Classification report - TREE \n")
    arquivo_accuracy_tree.write("\n Avaliação por classe \n")
    arquivo_accuracy_tree.write(classification_report(y_teste, predictions,zero_division=1))
    np.savetxt(arquivo_accuracy_history_tree, performance_history_tree,delimiter=",")
    np.savetxt(arquivo_f1_score_history_tree, f1_score_history_tree)
    np.savetxt(arquivo_tempo_history_tree,tempo_history_tree,fmt="%s")
    arquivo_accuracy_tree.close()
    
def committee_sampling_mlp(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial):
    
    from sklearn.neural_network import MLPClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
    import functools
    from IPython.display import clear_output
    from modAL.uncertainty import classifier_uncertainty
    from modAL.models import ActiveLearner, Committee
    from modAL.uncertainty import uncertainty_sampling
    from modAL.disagreement import vote_entropy_sampling
    from functools import partial
    
    
        
    #define nome de arquivos para salvar
    indica_pool=str(idx_dobra)
    
    # recupera as amostras de treino iniciais - a extratificação realizada só serve para tal finalidade.
    # No caso força a buscar pelo menos uma amostras de cada rótulo disponível (train_size= len(np.unique(y_raw)).
    # Realizar a busca aleatoriamente não garante iniciar com uma instância de cada classe.
    X_train_inicial, X_test_inicial, y_train_inicial, y_test_inicial = train_test_split(X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]], train_size= len(np.unique(y_raw[idx_data[idx_dobra][TRAIN]])) + t_inicial, random_state=42, stratify = y_raw[idx_data[idx_dobra][TRAIN]])
    #print("tamanho de X_train inicial: ",X_train_inicial.shape," tamanho de y_train inicial: ",y_train_inicial.shape)
    #print(y_train_inicial)  
    # recupera amostras de teste de acordo com a dobra em uso
    X_teste, y_teste = X_raw[idx_data[idx_dobra][TEST]], y_raw[idx_data[idx_dobra][TEST]]
    # recupera amostras de treino (será o pool) de acordo com a dobra em uso
    X_pool, y_pool = X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]]
    
    #isola exemplos rotulados para o treinamento inicial
    X_train = X_train_inicial
    y_train = y_train_inicial
    
    # initializing Committee members
    n_members = 3
    learner_list = list()
    
    #instancia classificadores de aprendizado ativo
    preset_batch = partial(vote_entropy_sampling,n_instances=BATCH_SIZE)
    learner_mlp = ActiveLearner(estimator=MLPClassifier(max_iter = 2000),X_training=X_train, y_training=y_train)
    learner_list.append(learner_mlp)
    arquivo_accuracy_mlp = open("committee_accuracy_performance_mlp_dobra_"+indica_pool+".txt","a")
    arquivo_accuracy_history_mlp = ("committee_accuracy_history_mlp_dobra_"+indica_pool+".csv")
    arquivo_f1_score_history_mlp = ("committee_F1_Score_history_mlp_dobra_"+indica_pool+".csv")
    arquivo_tempo_history_mlp = ("committee_tempo_history_mlp_dobra_"+indica_pool+".csv")
              
    # assembling the committee
    committee = Committee(learner_list=learner_list,query_strategy=preset_batch)
    
    #Registro da pontuação na porção de teste com o treinamento inicial
    performance_history_mlp=[]
    f1_score_history_mlp=[]
    tempo_history_mlp=[]
    uncertain_sample_score_mlp = committee.score(X_teste, y_teste)
    performance_history_mlp.append(uncertain_sample_score_mlp)
    predictions = committee.predict(X_teste)
    f1score_mlp = 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
    f1_score_history_mlp.append(f1score_mlp)
    
        
    for index in range(N_QUERIES):
        #inicia calculo do tempo de processamento da estratégia (consulta e inserção)
        t1_mlp = time.time()
        #recupera amostras do pool baseado na estratégia de consulta
        query_index, query_instance = committee.query(X_pool)
            
        # Ensina ao modelo ActiveLearner o registro solicitado (amostras vão para o topo).
        committee.teach(X=X_pool[query_index].reshape(BATCH_SIZE, -1),y=y_pool[query_index].reshape(BATCH_SIZE, ))
       
        # apaga os modelos consultados
        X_pool = np.delete(X_pool, query_index, axis=0)
        y_pool = np.delete(y_pool, query_index)
        
        #termina calculo de processamentoda estratégia
        t2_mlp = time.time()
        time_elapsed_mlp = (t2_mlp-t1_mlp)
        hours_mlp, rem_mlp = divmod(time_elapsed_mlp, 3600)
        minutes_mlp, seconds_mlp = divmod(rem_mlp, 60)
        tempo_history_mlp.append("{:0>2}:{:0>2}:{:0>2}".format(int(hours_mlp),int(minutes_mlp),int(seconds_mlp)))
        
        # verifica a performance após a inclusão de dados novos 
        committee_sample_score_mlp = committee.score(X_teste, y_teste)
        predictions = committee.predict(X_teste)
        clear_output(wait=True)
        print('Accuracy MLP after query no. %d: %f' % (index+1, committee_sample_score_mlp))
        arquivo_accuracy_mlp.write('Accuracy after query no. %d: %f \n' % (index+1,committee_sample_score_mlp))
        performance_history_mlp.append(committee_sample_score_mlp)
        #print ('Accuracy after query no. %d: %f' % (index+1, accuracy_score(y_test, predictions)))
        #arquivo_performance_mlp.write('Accuracy after query no. %d: %f \n' % (index+1,accuracy_score(y_teste, predictions)))
        #print ('Precision after query no. %d: %f' % (index+1, precision_score(y_test, predictions,average='macro',zero_division=1)))
        arquivo_accuracy_mlp.write('Precision after query no. %d: %f \n' % (index+1,precision_score(y_teste, predictions,average='macro',zero_division=1)))
        #print ('Recall after query no. %d: %f' % (index+1, recall_score(y_test, predictions, average='macro',zero_division=1)))
        arquivo_accuracy_mlp.write('Recall after query no. %d: %f \n' % (index+1, recall_score(y_teste, predictions, average='macro',zero_division=1)))
        #print ('F1 score after query no. %d: %f' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
        f1score_mlp= 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
        arquivo_accuracy_mlp.write('F1 score after query no. %d: %f \n' % (index+1, f1score_mlp))
        f1_score_history_mlp.append(f1score_mlp)
        #print ("========================================")
        arquivo_accuracy_mlp.write('======================================== \n')
                
    arquivo_accuracy_mlp.write("\n Classification report - MLP \n")
    arquivo_accuracy_mlp.write("\n Avaliação por classe \n")
    arquivo_accuracy_mlp.write(classification_report(y_teste, predictions,zero_division=1))
    np.savetxt(arquivo_accuracy_history_mlp, performance_history_mlp,delimiter=",")
    np.savetxt(arquivo_f1_score_history_mlp, f1_score_history_mlp)
    np.savetxt(arquivo_tempo_history_mlp,tempo_history_mlp,fmt="%s")
    arquivo_accuracy_mlp.close()
    
def committee_sampling_xgb(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial):
    
    from sklearn.neural_network import MLPClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
    import functools
    from IPython.display import clear_output
    from modAL.uncertainty import classifier_uncertainty
    from modAL.models import ActiveLearner, Committee
    from modAL.uncertainty import uncertainty_sampling
    from modAL.disagreement import vote_entropy_sampling
    from functools import partial
    
    
        
    #define nome de arquivos para salvar
    indica_pool=str(idx_dobra)
    
    # recupera as amostras de treino iniciais - a extratificação realizada só serve para tal finalidade.
    # No caso força a buscar pelo menos uma amostras de cada rótulo disponível (train_size= len(np.unique(y_raw)).
    # Realizar a busca aleatoriamente não garante iniciar com uma instância de cada classe.
    X_train_inicial, X_test_inicial, y_train_inicial, y_test_inicial = train_test_split(X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]], train_size= len(np.unique(y_raw[idx_data[idx_dobra][TRAIN]])) + t_inicial, random_state=42, stratify = y_raw[idx_data[idx_dobra][TRAIN]])
    #print("tamanho de X_train inicial: ",X_train_inicial.shape," tamanho de y_train inicial: ",y_train_inicial.shape)
    #print(y_train_inicial)  
    # recupera amostras de teste de acordo com a dobra em uso
    X_teste, y_teste = X_raw[idx_data[idx_dobra][TEST]], y_raw[idx_data[idx_dobra][TEST]]
    # recupera amostras de treino (será o pool) de acordo com a dobra em uso
    X_pool, y_pool = X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]]
    
    #isola exemplos rotulados para o treinamento inicial
    X_train = X_train_inicial
    y_train = y_train_inicial
    
    # initializing Committee members
    n_members = 3
    learner_list = list()
    
    #instancia classificadores de aprendizado ativo
    preset_batch = partial(vote_entropy_sampling,n_instances=BATCH_SIZE)
    learner_xgb = ActiveLearner(estimator=GradientBoostingClassifier(n_estimators=7, learning_rate=1.0,max_depth=1, random_state=42),X_training=X_train, y_training=y_train)
    learner_list.append(learner_xgb)
    arquivo_accuracy_xgb = open("committee_accuracy_performance_xgb_dobra_"+indica_pool+".txt","a")
    arquivo_accuracy_history_xgb = ("committee_accuracy_history_xgb_dobra_"+indica_pool+".csv")
    arquivo_f1_score_history_xgb = ("committee_F1_Score_history_xgb_dobra_"+indica_pool+".csv")
    arquivo_tempo_history_xgb = ("committee_tempo_history_xgb_dobra_"+indica_pool+".csv")
              
    # assembling the committee
    committee = Committee(learner_list=learner_list,query_strategy=preset_batch)
    
    #Registro da pontuação na porção de teste com o treinamento inicial
    performance_history_xgb=[]
    f1_score_history_xgb=[]
    tempo_history_xgb=[]
    uncertain_sample_score_xgb = committee.score(X_teste, y_teste)
    performance_history_xgb.append(uncertain_sample_score_xgb)
    predictions = committee.predict(X_teste)
    f1score_xgb = 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
    f1_score_history_xgb.append(f1score_xgb)
    
        
    for index in range(N_QUERIES):
        #inicia calculo do tempo de processamento da estratégia (consulta e inserção)
        t1_xgb = time.time()
        #recupera amostras do pool baseado na estratégia de consulta
        query_index, query_instance = committee.query(X_pool)
            
        # Ensina ao modelo ActiveLearner o registro solicitado (amostras vão para o topo).
        committee.teach(X=X_pool[query_index].reshape(BATCH_SIZE, -1),y=y_pool[query_index].reshape(BATCH_SIZE, ))
       
        # apaga os modelos consultados
        X_pool = np.delete(X_pool, query_index, axis=0)
        y_pool = np.delete(y_pool, query_index)
        
        #termina calculo de processamentoda estratégia
        t2_xgb = time.time()
        time_elapsed_xgb = (t2_xgb-t1_xgb)
        hours_xgb, rem_xgb = divmod(time_elapsed_xgb, 3600)
        minutes_xgb, seconds_xgb = divmod(rem_xgb, 60)
        tempo_history_xgb.append("{:0>2}:{:0>2}:{:0>2}".format(int(hours_xgb),int(minutes_xgb),int(seconds_xgb)))
        
        # verifica a performance após a inclusão de dados novos 
        committee_sample_score_xgb = committee.score(X_teste, y_teste)
        predictions = committee.predict(X_teste)
        clear_output(wait=True)
        print('Accuracy XGB after query no. %d: %f' % (index+1, committee_sample_score_xgb))
        arquivo_accuracy_xgb.write('Accuracy after query no. %d: %f \n' % (index+1,committee_sample_score_xgb))
        performance_history_xgb.append(committee_sample_score_xgb)
        #print ('Accuracy after query no. %d: %f' % (index+1, accuracy_score(y_test, predictions)))
        #arquivo_performance_xgb.write('Accuracy after query no. %d: %f \n' % (index+1,accuracy_score(y_teste, predictions)))
        #print ('Precision after query no. %d: %f' % (index+1, precision_score(y_test, predictions,average='macro',zero_division=1)))
        arquivo_accuracy_xgb.write('Precision after query no. %d: %f \n' % (index+1,precision_score(y_teste, predictions,average='macro',zero_division=1)))
        #print ('Recall after query no. %d: %f' % (index+1, recall_score(y_test, predictions, average='macro',zero_division=1)))
        arquivo_accuracy_xgb.write('Recall after query no. %d: %f \n' % (index+1, recall_score(y_teste, predictions, average='macro',zero_division=1)))
        #print ('F1 score after query no. %d: %f' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
        f1score_xgb= 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
        arquivo_accuracy_xgb.write('F1 score after query no. %d: %f \n' % (index+1, f1score_xgb))
        f1_score_history_xgb.append(f1score_xgb)
        #print ("========================================")
        arquivo_accuracy_xgb.write('======================================== \n')
                
    arquivo_accuracy_xgb.write("\n Classification report - XGB \n")
    arquivo_accuracy_xgb.write("\n Avaliação por classe \n")
    arquivo_accuracy_xgb.write(classification_report(y_teste, predictions,zero_division=1))
    np.savetxt(arquivo_accuracy_history_xgb, performance_history_xgb,delimiter=",")
    np.savetxt(arquivo_f1_score_history_xgb, f1_score_history_xgb)
    np.savetxt(arquivo_tempo_history_xgb,tempo_history_xgb,fmt="%s")
    arquivo_accuracy_xgb.close()
    
def committee_sampling_svm(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial):
    
    from sklearn.neural_network import MLPClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
    import functools
    from IPython.display import clear_output
    from modAL.uncertainty import classifier_uncertainty
    from modAL.models import ActiveLearner, Committee
    from modAL.uncertainty import uncertainty_sampling
    from modAL.disagreement import vote_entropy_sampling
    from functools import partial
    
    
        
    #define nome de arquivos para salvar
    indica_pool=str(idx_dobra)
    
    # recupera as amostras de treino iniciais - a extratificação realizada só serve para tal finalidade.
    # No caso força a buscar pelo menos uma amostras de cada rótulo disponível (train_size= len(np.unique(y_raw)).
    # Realizar a busca aleatoriamente não garante iniciar com uma instância de cada classe.
    X_train_inicial, X_test_inicial, y_train_inicial, y_test_inicial = train_test_split(X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]], train_size= len(np.unique(y_raw[idx_data[idx_dobra][TRAIN]])) + t_inicial, random_state=42, stratify = y_raw[idx_data[idx_dobra][TRAIN]])
    #print("tamanho de X_train inicial: ",X_train_inicial.shape," tamanho de y_train inicial: ",y_train_inicial.shape)
    #print(y_train_inicial)  
    # recupera amostras de teste de acordo com a dobra em uso
    X_teste, y_teste = X_raw[idx_data[idx_dobra][TEST]], y_raw[idx_data[idx_dobra][TEST]]
    # recupera amostras de treino (será o pool) de acordo com a dobra em uso
    X_pool, y_pool = X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]]
    
    #isola exemplos rotulados para o treinamento inicial
    X_train = X_train_inicial
    y_train = y_train_inicial
    
    # initializing Committee members
    n_members = 3
    learner_list = list()
    
    #instancia classificadores de aprendizado ativo
    preset_batch = partial(vote_entropy_sampling,n_instances=BATCH_SIZE)
    learner_svm = ActiveLearner(estimator=svm.SVC(kernel='linear',probability=True),X_training=X_train, y_training=y_train)
    learner_list.append(learner_svm)
    arquivo_accuracy_svm = open("committee_accuracy_performance_svm_dobra_"+indica_pool+".txt","a")
    arquivo_accuracy_history_svm = ("committee_accuracy_history_svm_dobra_"+indica_pool+".csv")
    arquivo_f1_score_history_svm = ("committee_F1_Score_history_svm_dobra_"+indica_pool+".csv")
    arquivo_tempo_history_svm = ("committee_tempo_history_svm_dobra_"+indica_pool+".csv")
              
    # assembling the committee
    committee = Committee(learner_list=learner_list,query_strategy=preset_batch)
    
    #Registro da pontuação na porção de teste com o treinamento inicial
    performance_history_svm=[]
    f1_score_history_svm=[]
    tempo_history_svm=[]
    uncertain_sample_score_svm = committee.score(X_teste, y_teste)
    performance_history_svm.append(uncertain_sample_score_svm)
    predictions = committee.predict(X_teste)
    f1score_svm = 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
    f1_score_history_svm.append(f1score_svm)
    
        
    for index in range(N_QUERIES):
        #inicia calculo do tempo de processamento da estratégia (consulta e inserção)
        t1_svm = time.time()
        #recupera amostras do pool baseado na estratégia de consulta
        query_index, query_instance = committee.query(X_pool)
            
        # Ensina ao modelo ActiveLearner o registro solicitado (amostras vão para o topo).
        committee.teach(X=X_pool[query_index].reshape(BATCH_SIZE, -1),y=y_pool[query_index].reshape(BATCH_SIZE, ))
       
        # apaga os modelos consultados
        X_pool = np.delete(X_pool, query_index, axis=0)
        y_pool = np.delete(y_pool, query_index)
        
        #termina calculo de processamentoda estratégia
        t2_svm = time.time()
        time_elapsed_svm = (t2_svm-t1_svm)
        hours_svm, rem_svm = divmod(time_elapsed_svm, 3600)
        minutes_svm, seconds_svm = divmod(rem_svm, 60)
        tempo_history_svm.append("{:0>2}:{:0>2}:{:0>2}".format(int(hours_svm),int(minutes_svm),int(seconds_svm)))
        
        # verifica a performance após a inclusão de dados novos 
        committee_sample_score_svm = committee.score(X_teste, y_teste)
        predictions = committee.predict(X_teste)
        clear_output(wait=True)
        print('Accuracy SVM after query no. %d: %f' % (index+1, committee_sample_score_svm))
        arquivo_accuracy_svm.write('Accuracy after query no. %d: %f \n' % (index+1,committee_sample_score_svm))
        performance_history_svm.append(committee_sample_score_svm)
        #print ('Accuracy after query no. %d: %f' % (index+1, accuracy_score(y_test, predictions)))
        #arquivo_performance_svm.write('Accuracy after query no. %d: %f \n' % (index+1,accuracy_score(y_teste, predictions)))
        #print ('Precision after query no. %d: %f' % (index+1, precision_score(y_test, predictions,average='macro',zero_division=1)))
        arquivo_accuracy_svm.write('Precision after query no. %d: %f \n' % (index+1,precision_score(y_teste, predictions,average='macro',zero_division=1)))
        #print ('Recall after query no. %d: %f' % (index+1, recall_score(y_test, predictions, average='macro',zero_division=1)))
        arquivo_accuracy_svm.write('Recall after query no. %d: %f \n' % (index+1, recall_score(y_teste, predictions, average='macro',zero_division=1)))
        #print ('F1 score after query no. %d: %f' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
        f1score_svm= 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
        arquivo_accuracy_svm.write('F1 score after query no. %d: %f \n' % (index+1, f1score_svm))
        f1_score_history_svm.append(f1score_svm)
        #print ("========================================")
        arquivo_accuracy_svm.write('======================================== \n')
                
    arquivo_accuracy_svm.write("\n Classification report - SVM \n")
    arquivo_accuracy_svm.write("\n Avaliação por classe \n")
    arquivo_accuracy_svm.write(classification_report(y_teste, predictions,zero_division=1))
    np.savetxt(arquivo_accuracy_history_svm, performance_history_svm,delimiter=",")
    np.savetxt(arquivo_f1_score_history_svm, f1_score_history_svm)
    np.savetxt(arquivo_tempo_history_svm,tempo_history_svm,fmt="%s")
    arquivo_accuracy_svm.close()
    
def committee_sampling_nb(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial):
    
    from sklearn.neural_network import MLPClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
    import functools
    from IPython.display import clear_output
    from modAL.uncertainty import classifier_uncertainty
    from modAL.models import ActiveLearner, Committee
    from modAL.uncertainty import uncertainty_sampling
    from modAL.disagreement import vote_entropy_sampling
    from functools import partial
    
    
        
    #define nome de arquivos para salvar
    indica_pool=str(idx_dobra)
    
    # recupera as amostras de treino iniciais - a extratificação realizada só serve para tal finalidade.
    # No caso força a buscar pelo menos uma amostras de cada rótulo disponível (train_size= len(np.unique(y_raw)).
    # Realizar a busca aleatoriamente não garante iniciar com uma instância de cada classe.
    X_train_inicial, X_test_inicial, y_train_inicial, y_test_inicial = train_test_split(X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]], train_size= len(np.unique(y_raw[idx_data[idx_dobra][TRAIN]])) + t_inicial, random_state=42, stratify = y_raw[idx_data[idx_dobra][TRAIN]])
    #print("tamanho de X_train inicial: ",X_train_inicial.shape," tamanho de y_train inicial: ",y_train_inicial.shape)
    #print(y_train_inicial)  
    # recupera amostras de teste de acordo com a dobra em uso
    X_teste, y_teste = X_raw[idx_data[idx_dobra][TEST]], y_raw[idx_data[idx_dobra][TEST]]
    # recupera amostras de treino (será o pool) de acordo com a dobra em uso
    X_pool, y_pool = X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]]
    
    #isola exemplos rotulados para o treinamento inicial
    X_train = X_train_inicial
    y_train = y_train_inicial
    
    # initializing Committee members
    n_members = 3
    learner_list = list()
    
    #instancia classificadores de aprendizado ativo
    preset_batch = partial(vote_entropy_sampling,n_instances=BATCH_SIZE)
    learner_nb = ActiveLearner(estimator=GaussianNB(),X_training=X_train, y_training=y_train)
    learner_list.append(learner_nb)
    arquivo_accuracy_nb = open("committee_accuracy_performance_nb_dobra_"+indica_pool+".txt","a")
    arquivo_accuracy_history_nb = ("committee_accuracy_history_nb_dobra_"+indica_pool+".csv")
    arquivo_f1_score_history_nb = ("committee_F1_Score_history_nb_dobra_"+indica_pool+".csv")
    arquivo_tempo_history_nb = ("committee_tempo_history_nb_dobra_"+indica_pool+".csv")
              
    # assembling the committee
    committee = Committee(learner_list=learner_list,query_strategy=preset_batch)
    
    #Registro da pontuação na porção de teste com o treinamento inicial
    performance_history_nb=[]
    f1_score_history_nb=[]
    tempo_history_nb=[]
    uncertain_sample_score_nb = committee.score(X_teste, y_teste)
    performance_history_nb.append(uncertain_sample_score_nb)
    predictions = committee.predict(X_teste)
    f1score_nb = 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
    f1_score_history_nb.append(f1score_nb)
    
        
    for index in range(N_QUERIES):
        #inicia calculo do tempo de processamento da estratégia (consulta e inserção)
        t1_nb = time.time()
        #recupera amostras do pool baseado na estratégia de consulta
        query_index, query_instance = committee.query(X_pool)
            
        # Ensina ao modelo ActiveLearner o registro solicitado (amostras vão para o topo).
        committee.teach(X=X_pool[query_index].reshape(BATCH_SIZE, -1),y=y_pool[query_index].reshape(BATCH_SIZE, ))
       
        # apaga os modelos consultados
        X_pool = np.delete(X_pool, query_index, axis=0)
        y_pool = np.delete(y_pool, query_index)
        
        #termina calculo de processamentoda estratégia
        t2_nb = time.time()
        time_elapsed_nb = (t2_nb-t1_nb)
        hours_nb, rem_nb = divmod(time_elapsed_nb, 3600)
        minutes_nb, seconds_nb = divmod(rem_nb, 60)
        tempo_history_nb.append("{:0>2}:{:0>2}:{:0>2}".format(int(hours_nb),int(minutes_nb),int(seconds_nb)))
        
        # verifica a performance após a inclusão de dados novos 
        committee_sample_score_nb = committee.score(X_teste, y_teste)
        predictions = committee.predict(X_teste)
        clear_output(wait=True)
        print('Accuracy NB after query no. %d: %f' % (index+1, committee_sample_score_nb))
        arquivo_accuracy_nb.write('Accuracy after query no. %d: %f \n' % (index+1,committee_sample_score_nb))
        performance_history_nb.append(committee_sample_score_nb)
        #print ('Accuracy after query no. %d: %f' % (index+1, accuracy_score(y_test, predictions)))
        #arquivo_performance_nb.write('Accuracy after query no. %d: %f \n' % (index+1,accuracy_score(y_teste, predictions)))
        #print ('Precision after query no. %d: %f' % (index+1, precision_score(y_test, predictions,average='macro',zero_division=1)))
        arquivo_accuracy_nb.write('Precision after query no. %d: %f \n' % (index+1,precision_score(y_teste, predictions,average='macro',zero_division=1)))
        #print ('Recall after query no. %d: %f' % (index+1, recall_score(y_test, predictions, average='macro',zero_division=1)))
        arquivo_accuracy_nb.write('Recall after query no. %d: %f \n' % (index+1, recall_score(y_teste, predictions, average='macro',zero_division=1)))
        #print ('F1 score after query no. %d: %f' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
        f1score_nb= 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
        arquivo_accuracy_nb.write('F1 score after query no. %d: %f \n' % (index+1, f1score_nb))
        f1_score_history_nb.append(f1score_nb)
        #print ("========================================")
        arquivo_accuracy_nb.write('======================================== \n')
                
    arquivo_accuracy_nb.write("\n Classification report - NB \n")
    arquivo_accuracy_nb.write("\n Avaliação por classe \n")
    arquivo_accuracy_nb.write(classification_report(y_teste, predictions,zero_division=1))
    np.savetxt(arquivo_accuracy_history_nb, performance_history_nb,delimiter=",")
    np.savetxt(arquivo_f1_score_history_nb, f1_score_history_nb)
    np.savetxt(arquivo_tempo_history_nb,tempo_history_nb,fmt="%s")
    arquivo_accuracy_nb.close()
    

import time
import sys
import threading
from datetime import datetime
from datetime import date
from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit, train_test_split
from timeit import Timer
import time
import functools
# importing the multiprocessing module
import multiprocessing

#inicia relogio
t1 = time.time()
# Define o tamanho das divisões feitas no dataset (cross-validation)
n_dobras = 10
# Define Tamanho inicial da amostra (toda estratégia parte de um tamanho mínimo aleatório).
t_inicial = 10
#Define contador de amostras utilizadas pela estratégia
sample_size = 0 

#define array de indices das partições
idx_data =[]
# cross validation bags - n_splits
data_cv = StratifiedShuffleSplit(n_splits= n_dobras,train_size=22000,test_size=22000,random_state=42) 
data_cv.get_n_splits(X_raw, y_raw)
# chame a instância e gere os dados sobre a base original
type(data_cv.split(X_raw, y_raw))
# dividir os dados - A função split.split () retorna índices para amostras de treino e amostras de teste. 
# Ele examinará o número de validação cruzada especificado e retornará cada vez que treinar 
# e testar os índices de amostra usando os conjuntos de dados de treinamento e teste que podem 
# ser criados filtrando o conjunto de dados inteiro. Por exemplo idx_data[0][1], o primeiro indice faz referencia
# a dobra e o segundo indice faz referencia a posição da dobra (0 = treino e 1 = teste). Logo TRAIN=0 e TEST=1
for train_index, test_index in data_cv.split(X_raw,y_raw):
    #print("TRAIN:", train_index, "TEST:", test_index)
    #print("n_split",n_splits,"TRAIN:", train_index, "TEST:", test_index)
    idx_data.append([train_index, test_index])
#verifica tamanho das dobras (numero de instâncias de cada dobra)
print("tamanho de cada dobra: ",idx_data[3][0].shape)

TRAIN =0
TEST =1

# DEFINE NUMERO DE QUERIES
BATCH_SIZE = 200
N_RAW_SAMPLES = 10000
N_QUERIES = N_RAW_SAMPLES // BATCH_SIZE


for idx_dobra in range(n_dobras):
    if __name__ == "__main__":
        # criando os processos
        #p1 = multiprocessing.Process(target=committee_sampling_knn(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial))
        #p2 = multiprocessing.Process(target=committee_sampling_rf(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial))
        #p3 = multiprocessing.Process(target=committee_sampling_nb(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial))
        #p4 = multiprocessing.Process(target=committee_sampling_tree(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial))
        #p5 = multiprocessing.Process(target=committee_sampling_mlp(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial))
        #p6 = multiprocessing.Process(target=committee_sampling_xgb(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial))
        #p7 = multiprocessing.Process(target=committee_sampling_svm(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial))
        
        # iniciando os processos
        #p1.start()
        #p2.start()
        #p3.start()
        #p4.start()
        #p5.start()
        #p6.start()
        #p7.start()
        
        # aguardando os processos serem finalizados
        #p1.join()
        #p2.join()
        #p3.join()
        #p4.join()
        #p5.join()
        #p6.join()
        #p7.join
        
        # todos os processos finalizados
        print("Terminado!")

    
t2 = time.time()
time_elapsed = (t2-t1)
hours, rem = divmod(time_elapsed, 3600)
minutes, seconds = divmod(rem, 60)
print("Tempo total: {:0>2}:{:0>2}:{:0>2}".format(int(hours),int(minutes),int(seconds)))